In [2]:
!pip install tensorflow

In [3]:
import tensorflow as tf

In [4]:
print(tf.__version__)

2.17.0


In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [6]:
dataset = pd.read_csv('Churn_Modelling.csv')

In [7]:
dataset.shape

(10000, 14)

In [8]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [9]:
dataset.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [10]:
dataset.Exited.value_counts()   #dataset is not looking balanced

Exited
0    7963
1    2037
Name: count, dtype: int64

In [70]:
#Divinding the dataset into independent and dependent features

X=dataset.iloc[:,3:13]  #not taking first three cols and last col also
y=dataset.iloc[:,13]    #dependent feature (exited)

In [72]:
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [74]:
y.head()

0    1
1    0
2    1
3    0
4    0
Name: Exited, dtype: int64

## FEATURE ENGINEERING

In [78]:
#so we have two categorical features in dataset which we need to handle in feature engineering

In [80]:
dataset.Geography.value_counts()

Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64

In [82]:
geography = pd.get_dummies(X.Geography,drop_first=True)   ##converting categorical into int by One-hot encoding

In [84]:
geography=geography.astype(int)

In [86]:
geography

,Germany,Spain
0,0,0
1,0,1
2,0,0
3,0,0
4,0,1
...,...,...
9995,0,0
9996,0,0
9997,0,0
9998,1,0


In [88]:
gender = pd.get_dummies(X.Gender,drop_first=True)   #converting categorical into int by One-hot encoding

In [90]:
gender=gender.astype(int)

In [92]:
gender

,Male
0,0
1,0
2,0
3,0
4,0
...,...
9995,1
9996,1
9997,0
9998,1


In [94]:
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [96]:
X=X.drop(['Geography','Gender'],axis=1)

In [98]:
X.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,42,2,0.00,1,1,1,101348.88
1,608,41,1,83807.86,1,0,1,112542.58
2,502,42,8,159660.80,3,1,0,113931.57
3,699,39,1,0.00,2,0,0,93826.63
4,850,43,2,125510.82,1,1,1,79084.10


In [100]:
## concatenate these variables with dataframe
X=pd.concat([X,geography,gender],axis=1)

In [102]:
X.head()  #now all features in int

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Germany,Spain,Male
0,619,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,1,0


In [117]:
#Splitting dataset into Training and Test Set

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)  #dataset got splitted 

In [127]:
#feature scaling
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)   #to avoid data lekage
X_test=sc.transform(X_test)

#Here, sc.fit_transform(X_train) standardizes the training data by computing the mean and standard deviation of the training features, and then scales them. 
#When you later transform the test data, you use the same statistics learned from the training set to maintain consistency.

What is fit_transform()?

fit(): Calculates the necessary statistics (e.g., mean, standard deviation) based on the training data. This is done only on the training set so that future data (e.g., test set) is transformed using the same parameters.

transform(): Applies the transformation using the statistics calculated in the fit() step.
    
fit_transform(): Combines both fit() and transform() into one method for convenience.

In [121]:
X_train   #got standardized

array([[ 0.16958176, -0.46460796,  0.00666099, ..., -0.5698444 ,
         1.74309049, -1.09168714],
       [-2.30455945,  0.30102557, -1.37744033, ...,  1.75486502,
        -0.57369368,  0.91601335],
       [-1.19119591, -0.94312892, -1.031415  , ..., -0.5698444 ,
        -0.57369368, -1.09168714],
       ...,
       [ 0.9015152 , -0.36890377,  0.00666099, ..., -0.5698444 ,
        -0.57369368,  0.91601335],
       [-0.62420521, -0.08179119,  1.39076231, ..., -0.5698444 ,
         1.74309049, -1.09168714],
       [-0.28401079,  0.87525072, -1.37744033, ...,  1.75486502,
        -0.57369368, -1.09168714]])

In [123]:
X_train.shape

(8000, 11)

In [125]:
X_test.shape

(2000, 11)

## Part 2:  Now let's create ANN Model

In [148]:
import tensorflow as tf
from tensorflow.keras import Sequential  #for creating one block of neural network
from tensorflow.keras.layers import Dense, Dropout  #Dense for creating neurons for different layers
from tensorflow.keras.layers import LeakyReLU,ReLU,PReLU,ELU  #activation functions 


In [150]:
#Lets intialize the ANN Model
classifier=Sequential()

In [199]:
#Adding the input layer 
classifier.add(Dense(units=11,activation='relu')) #Dense is used to create layers(the neurons) and units=11 means total 11 neurons

In [201]:
#adding the first hidden layer
classifier.add(Dense(units=7,activation='relu'))
classifier.add(Dropout(0.2))  #20% neurons will be dead while trainig

In [203]:
#adding the second hidden layer
classifier.add(Dense(units=6,activation='relu'))
classifier.add(Dropout(0.3))  #30% neurons will be dead while trainig

In [205]:
 #adding the output layer
classifier.add(Dense(units=1,activation='sigmoid'))

In [207]:
#Compile the Model
classifier.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
#accuracy, to evaluate the model performance during training.

In [209]:
#early stopping means if my val_loss not improves then we will be stopping the epochs over there
import tensorflow as tf
early_stopping=tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=0,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0,
)

In [211]:
# Train the model
model_history = classifier.fit(X_train, y_train, epochs=1000, batch_size=10, validation_data=(X_test, y_test),callbacks=early_stopping)
#early stopping means if my val_loss not improves then it will be stopping the epochs over there automatically

Epoch 1/1000
800/800 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - accuracy: 0.8081 - loss: 0.5258 - val_accuracy: 0.8255 - val_loss: 0.3717
Epoch 2/1000
800/800 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8359 - loss: 0.3924 - val_accuracy: 0.8560 - val_loss: 0.3567
Epoch 3/1000
800/800 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8407 - loss: 0.3950 - val_accuracy: 0.8590 - val_loss: 0.3615


In [212]:
#so after 3 epochs we got the best fit and best accuracy and best min  validation loss  and it got stopped automatically

In [213]:
model_history.history.keys()

dict_keys(['accuracy', 'loss', 'val_accuracy', 'val_loss'])

In [214]:
#Testing or evaluating the model

#Predicting the test set results
y_pred = classifier.predict(X_test)
y_pred=(y_pred>=0.5)

63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step


In [215]:
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [216]:
#Calculate the accuracy score

from sklearn.metrics import accuracy_score
score=accuracy_score(y_pred,y_test)

In [217]:
score  #this the accuracy of our model 

0.859

In [218]:
#get the weights for each input which we are passing
classifier.get_weights()

[array([[-0.15045008, -0.06844289, -0.05446208,  0.09899326, -0.51348484,
          0.19525567,  0.42643765,  0.1612222 , -0.11428823, -0.54919416,
          0.15183207],
        [-0.34767607, -0.6005198 ,  0.5186709 ,  0.68903106, -0.2924157 ,
         -1.1936593 , -0.00894766,  2.0346973 ,  0.18878196,  1.1078634 ,
          0.42848608],
        [-0.36648598,  0.18921119,  0.11752292, -0.08316278, -0.5786493 ,
         -0.6234558 , -0.82115   , -0.45606568, -0.1098296 , -0.37247723,
          0.30358657],
        [ 1.7031143 ,  0.63606006,  0.23000649, -1.133065  , -0.64386034,
         -0.6739249 ,  0.074344  ,  0.37250403,  1.3324108 , -0.86679417,
         -0.73787314],
        [-0.13926457, -0.27201506,  1.7418886 , -1.1855439 , -0.5251962 ,
          1.0084344 , -0.8693175 , -0.43418247,  1.0910137 , -2.0822663 ,
          0.34126428],
        [ 0.17504863, -0.6021081 ,  0.00624143,  0.1443235 , -0.47779176,
          0.55011296,  0.4460598 , -0.31906265, -0.0021095 , -0.2410438